In [1]:
import numpy as np
from collections import Counter
import time

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [4]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

In [5]:
class minmax_alphabeta:
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):
        self.board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
        self.player_turn = -1
    
    def check_end(self):
        if four_in_a_row(self.board, 1):
            # Alice won
            return 1
        elif four_in_a_row(self.board, -1):
            # Bob won
            return -1
        elif self.board.all():
            #IT IS A TIE
            return 0
        return None

    def max(self, alpha, beta, depth):
        result = self.check_end()
        if result != None:
            if result == 1:
                return (1/depth, 0)
            elif result == -1:
                return (-1/depth, 0)
            elif result == 0:
                return (0, 0)
        max_value = -2
        max_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, 1)
            (value, min_idx) = self.min(alpha, beta, depth+1)
            if value > max_value:
                max_value = value
                max_idx = i
            take_back(self.board, i)
            if max_value >= beta:
                return (max_value, max_idx)
            if max_value > alpha:
                alpha = max_value
        return (max_value, max_idx)
            
    
    def min(self, alpha, beta, depth):
        result = self.check_end()
        if result != None:
            if result == 1:
                return (1/depth, 0)
            elif result == -1:
                return (-1/depth, 0)
            elif result == 0:
                return (0, 0)
        min_value = 2
        min_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, -1)
            (value, max_idx) = self.max(alpha, beta, depth+1)
            if value < min_value:
                min_value = value
                min_idx = i
            take_back(self.board, i)
            if min_value <= alpha:
                return (min_value, min_idx)
            if min_value < beta:
                beta = min_value
        return (min_value, min_idx)
    
    def play(self):
        while True:
            print(self.board)
            self.result = self.check_end()
            if self.result != None:
                if self.result == 1:
                    print('The winner is Player 1')
                elif self.result == -1:
                    print('The winner is Player -1')
                elif self.result == 0:
                    print("It's a tie!")
                self.initialize_game()
                return
            if self.player_turn == -1:
                (m, index) = self.min(-2, 2, 1)
                play(self.board, index, -1)
                self.player_turn = 1

            # If it's AI's turn
            else:
                (m, index) = self.max(-2, 2, 1)
                play(self.board, index, 1)
                self.player_turn = -1

In [6]:
class minmax_alphabeta_mc:
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):
        self.board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
        self.player_turn = 1
    
    def check_end(self):
        if four_in_a_row(self.board, 1):
            # Alice won
            return 1
        elif four_in_a_row(self.board, -1):
            # Bob won
            return -1
        elif self.board.all():
            #IT IS A TIE
            return 0
        return None

    def max(self, alpha, beta, depth):
        result = self.check_end()
        if result != None or depth<=0:
            if result == 1:
                return (1, 0)
            elif result == -1:
                return (-1, 0)
            elif result == 0:
                return (0, 0)
            else:
                return (eval_board(self.board, 1), 0)
        max_value = -2
        max_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, 1)
            (value, min_idx) = self.min(alpha, beta, depth-1)
            if value > max_value:
                max_value = value
                max_idx = i
            take_back(self.board, i)
            if max_value >= beta:
                return (max_value, max_idx)
            if max_value > alpha:
                alpha = max_value
        return (max_value, max_idx)
            
    
    def min(self, alpha, beta, depth):
        result = self.check_end()
        if result != None or depth<=0:
            if result == 1:
                return (1, 0)
            elif result == -1:
                return (-1, 0)
            elif result == 0:
                return (0, 0)
            else:
                return (eval_board(self.board, -1), 0)
        min_value = 2
        min_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, -1)
            (value, max_idx) = self.max(alpha, beta, depth-1)
            if value < min_value:
                min_value = value
                min_idx = i
            take_back(self.board, i)
            if min_value <= alpha:
                return (min_value, min_idx)
            if min_value < beta:
                beta = min_value
        return (min_value, min_idx)

    def play(self, depth):
        turn =1
        while True:
            start_time = time.time()
            print('Turn', turn , 'Player:',self.player_turn)
            turn += 1
            self.result = self.check_end()
            if self.result != None:
                if self.result == 1:
                    print('The winner is Player 1')
                elif self.result == -1:
                    print('The winner is Player -1')
                elif self.result == 0:
                    print("It's a tie!")
                self.initialize_game()
                return
            if self.player_turn == -1:
                (m, index) = self.min(-2, 2, depth)
                play(self.board, index, -1)
                self.player_turn = 1
            else:
                (m, index) = self.max(-2, 2, depth)
                play(self.board, index, 1)
                self.player_turn = -1
            print(self.board)
            print("%.2f seconds" % (time.time() - start_time))

In [7]:
g = minmax_alphabeta_mc()
g.play(2)

Turn 1 Player: 1
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
30.56 seconds
Turn 2 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
31.84 seconds
Turn 3 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
22.05 seconds
Turn 4 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
18.79 seconds
Turn 5 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
17.02 seconds
Turn 6 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  1 -1  0  0  0]
 [ 1 -1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 